# Image-emotion project (from HCI course)

## Requirements import

In [ ]:
# importing required components
import os,sys
import glob, time, json
import urllib
import random
import torch
import torchvision
import numpy as np
import pandas as pd
# import seaborn as sns

# import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
# from imutils import paths
# from tqdm import tqdm_notebook as tqdm
# from keras.models import Model
# from keras.utils import np_utils
from PIL import Image
# from pdf2image import convert_from_path
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

## Environment settings

In [ ]:
# #Setting torch environment

# if torch.cuda.is_available():
#     DEVICE = torch.device('cuda')
# else:
#     DEVICE = torch.device('cpu')

# print('Using PyTorch version:', torch.__version__, ' Device: ', DEVICE)

In [ ]:
DEVICE = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' Device: ', DEVICE)

In [ ]:
class Args:
    # arugments
    epochs=10
    bs=16
    lr=0.001
    momentum=0.9
    num_channels=3
    num_classes=6
    verbose='store_true'
    seed=412

args = Args()

np.random.seed(args.seed)
random.seed(args.seed)
torch.manual_seed(args.seed)

In [ ]:
model_resnet18 = models.resnet18(pretrained=True).to(DEVICE)
## resnet 구조는 마지막 fc layer의 out_features 를 바꿔주면 되고.
model_resnet18.fc = nn.Linear(in_features = 512, out_features = args.num_classes).to(DEVICE)

In [ ]:
model = model_resnet18.to(DEVICE)

In [ ]:
# Data Transformation
data_transforms = transforms.Compose([
    transforms.Resize((227,227)),
    # transforms.Resize((256,256)),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomVerticalFlip(),
#     transforms.ColorJitter(contrast=(0.3, 1), saturation=(0.3, 1)),
    transforms.ToTensor()
])

## Training data load

In [ ]:
# Uploading image data/content/drive/MyDrive/Research_personal/image_emotion/images
emotion_data = torchvision.datasets.ImageFolder(root = 'E:/RESEARCH/Datasets/image_emotion/total/', transform = data_transforms)

In [ ]:
train_size = int(0.8 * len(emotion_data))
test_size = len(emotion_data)-train_size
print(train_size)
print(test_size)

In [ ]:
train_dataset, test_dataset = torch.utils.data.random_split(emotion_data, [train_size, test_size])

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.bs, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.bs, shuffle=False, num_workers=4)

In [ ]:
dataiter = iter(train_loader)
images, labels = next(dataiter)
print(labels)

In [ ]:
# Setting Optimizer and Objective Function

criterion = nn.CrossEntropyLoss() ## setup the loss function
optimizer = torch.optim.Adam(model.parameters(), lr = args.lr)
# scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=lambda epoch: 0.95 ** args.epochs)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, total_steps=50,anneal_strategy='cos')

# print(model)

## Model training

In [ ]:
# Function for checking model performance during CNN model

def train(model, train_loader, optimizer, log_interval):
    model.train()
    print(optimizer.param_groups[0]['lr'])

    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image),
                len(train_loader.dataset), 100. * batch_idx / len(train_loader),
                loss.item()))

#     scheduler.step() #for learning rate scheduler

In [ ]:
# Function for checking model performance during the learning process

def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    validation =[]

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()


    test_loss /= (len(test_loader))
    validation_accuracy = 100. * correct / len(test_loader.dataset)
    validation.append(validation_accuracy)

    return test_loss, validation_accuracy

In [ ]:
# Checking train, val loss and accuracy

los_total = []
acc_total = []

for epoch in range(1, args.epochs):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, validation_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tValidation Accuracy: {:.2f} % \n".format(
        epoch, test_loss, validation_accuracy))

    los_total.append(test_loss)
    acc_total.append(validation_accuracy)

## Save the trained model weights

In [ ]:
## 이렇게 해야 완전하게 모델 저장이 됨. (weight 외의 항목들도)
torch.save(model.state_dict(), 'E:/RESEARCH/Datasets/image_emotion/model_weights_six.pth')
# torch.save(model.state_dict(), 'E:/RESEARCH/Datasets/image_emotion/model_weights_four.pth')

In [ ]:
# model.load_state_dict(torch.load('E:/RESEARCH/Datasets/image_emotion/model_weights.pth'))

## Inference check

In [ ]:
classes = ('anger', 'happiness', 'sadness', 'fear')

In [ ]:
sample = Image.open('E:/RESEARCH/Datasets/image_emotion/sample1.jpg')

In [ ]:
sample_t = data_transforms(sample).to(DEVICE)
batch_t = torch.unsqueeze(sample_t, 0)
out = model(batch_t).to(DEVICE)
model.eval()
_, predicted = torch.sort(out, descending = True)
print([(classes[idx]) for idx in predicted[0][:1]])

## Summarize

In [ ]:
def image_emotion(data, model):
    ## get data
    img = data
    ## data transform for training
    transform = transforms.Compose([
        transforms.Resize((227, 227)),
        transforms.ToTensor()
    ])
    img_t = transform(img).to(DEVICE)
    ## resize to batch
    batch_t = torch.unsqueeze(img_t, 0)
    ## set the classes
    classes = ('anger', 'fear', 'sadness', 'happiness')
    ## model evaluate
    model.eval() 
    ## inference
    out = model(batch_t).to(DEVICE)
    _, predicted = torch.sort(out, descending = True)
    print("predicted emotion is:", [(classes[idx]) for idx in predicted[0][:1]])

In [ ]:
data = Image.open('E:/RESEARCH/Datasets/image_emotion/sample13.jpg')
model_ = models.resnet18(pretrained=True).to(DEVICE)
model_.fc = nn.Linear(in_features = 512, out_features = 4).to(DEVICE)
model = model_.to(DEVICE)
# model.load_state_dict(torch.load('D:/image_data/classify.pth'))
model.load_state_dict(torch.load('E:/RESEARCH/Datasets/image_emotion/model_weights.pth'))

In [ ]:
image_emotion(data, model)